In [1]:
import pandas as pd

df = pd.read_csv('data/sample_df.csv')
df.head()

,group_id,original_position,1,2,3,4,5,6,7,8,...,32,33,34,35,36,37,38,39,40,label
0,f061abd6-7eae-4bf4-bb50-88e80cc7c837,9,0.344828,0.015152,0.92,0.571429,0.478261,0.000000,0.000000,0.000000,...,0.010648,0.002064,0.001717,0.262489,0.267794,1.000000,0.013932,0.002017,0.967240,0
1,1669ecab-82b0-48b1-b429-7f5262204bfb,21,0.413793,0.090909,0.90,1.000000,0.217391,0.000000,0.000000,0.000000,...,0.014352,0.021672,0.001706,0.066377,0.097252,0.947368,0.013932,0.002129,0.944403,0
2,ba05c944-9754-462c-af8e-29a29cb7fa95,10,0.310345,0.015152,0.90,1.000000,0.521739,0.000000,0.000000,0.000000,...,0.014815,0.017544,0.002637,0.199186,0.272375,0.553846,0.027864,0.007285,0.934003,0
3,ba05c944-9754-462c-af8e-29a29cb7fa95,18,0.327586,0.035354,0.96,1.000000,0.521739,0.000000,0.000000,0.000000,...,0.002778,0.004128,0.000747,0.138146,0.145877,1.000000,0.010836,0.001569,0.919672,0
4,da3e6fd8-6107-4b72-a8c5-f7647443b0a0,20,0.310345,0.101010,0.94,0.142857,0.608696,0.076923,0.083333,0.063927,...,0.052778,0.052632,0.064694,0.172554,0.958069,0.586466,0.181115,0.044716,0.881276,0


In [2]:
from cross_validation import train_test_split

In [3]:
X_train, y_train, qids_train, X_test, y_test, qids_test, training_ids, testing_ids = train_test_split(
                                                                                    df, 'group_id', 'label')

In [4]:
imp_feats = [str(i) for i in range(1, 41)]

In [5]:
from blindfold import BlindFold

In [6]:
ranker=BlindFold()

In [7]:
trained_model = ranker.fit(X_train[imp_feats], y_train, qids_train, X_test[imp_feats], y_test, qids_test)

[0]	validation_0-ndcg@10:0.62178	validation_1-ndcg@10:0.62510
[1]	validation_0-ndcg@10:0.64203	validation_1-ndcg@10:0.64352
[2]	validation_0-ndcg@10:0.63921	validation_1-ndcg@10:0.64156
[3]	validation_0-ndcg@10:0.64050	validation_1-ndcg@10:0.64203
[4]	validation_0-ndcg@10:0.64568	validation_1-ndcg@10:0.64344
[5]	validation_0-ndcg@10:0.64643	validation_1-ndcg@10:0.64210
[6]	validation_0-ndcg@10:0.65110	validation_1-ndcg@10:0.64646
[7]	validation_0-ndcg@10:0.65667	validation_1-ndcg@10:0.64881


/Users/gurudutttaluja/.conda/envs/pythonProject/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,
/Users/gurudutttaluja/.conda/envs/pythonProject/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[8]	validation_0-ndcg@10:0.65566	validation_1-ndcg@10:0.64906
[9]	validation_0-ndcg@10:0.65532	validation_1-ndcg@10:0.65052


In [8]:
from evaluation import ndcg_for_df

ndcg_for_df(df[df.group_id.isin(training_ids)], pred_column='original_position', 
            y_label='label', group_column='group_id', k=100)

0.4830068447783998

In [9]:
df["pred"] = trained_model.predict(df[imp_feats])
df["rank"] = df.groupby("group_id")["pred"].rank(method="dense", ascending=False)

In [10]:
ndcg_for_df(df[df.group_id.isin(training_ids)], pred_column='rank', 
            y_label='label', group_column='group_id', k=100)

0.5267837586925701